In [42]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
with open("/content/Franknestein.txt", encoding="utf8") as text_file:
    file = text_file.read()
print(file)

In [61]:
sequence_length = 100
BATCH_SIZE = 128
EPOCHS = 30

In [44]:
def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

In [45]:
text = tokenize_words(file)

In [47]:
n_chars = len(text)
vocab=''.join(sorted(set(text)))
n_unique_chars = len(vocab)

In [48]:
print("n_chars",n_chars)
print("_____________")
print("vocab",vocab)
print("_____________")
print("n_unique_chars",n_unique_chars)


n_chars 269995
_____________
vocab  0123456789_abcdefghijklmnopqrstuvwxyzæèéêô
_____________
n_unique_chars 43


In [49]:
# dictionary that converts characters to integers
char2int ={c:i for i ,c in enumerate (vocab)}
char2int

{' ': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 '_': 11,
 'a': 12,
 'b': 13,
 'c': 14,
 'd': 15,
 'e': 16,
 'f': 17,
 'g': 18,
 'h': 19,
 'i': 20,
 'j': 21,
 'k': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'q': 28,
 'r': 29,
 's': 30,
 't': 31,
 'u': 32,
 'v': 33,
 'w': 34,
 'x': 35,
 'y': 36,
 'z': 37,
 'æ': 38,
 'è': 39,
 'é': 40,
 'ê': 41,
 'ô': 42}

In [50]:
# dictionary that converts integers to characters
int2char={i : c for i ,c in enumerate (vocab)}
int2char

{0: ' ',
 1: '0',
 2: '1',
 3: '2',
 4: '3',
 5: '4',
 6: '5',
 7: '6',
 8: '7',
 9: '8',
 10: '9',
 11: '_',
 12: 'a',
 13: 'b',
 14: 'c',
 15: 'd',
 16: 'e',
 17: 'f',
 18: 'g',
 19: 'h',
 20: 'i',
 21: 'j',
 22: 'k',
 23: 'l',
 24: 'm',
 25: 'n',
 26: 'o',
 27: 'p',
 28: 'q',
 29: 'r',
 30: 's',
 31: 't',
 32: 'u',
 33: 'v',
 34: 'w',
 35: 'x',
 36: 'y',
 37: 'z',
 38: 'æ',
 39: 'è',
 40: 'é',
 41: 'ê',
 42: 'ô'}

In [51]:
# convert all text into integers
import numpy as np
encoded_text=np.array([char2int[c]for c in text])
encoded_text

array([27, 29, 26, ..., 26, 22, 30])

In [ ]:
# construct tf.data.Dataset object
import tensorflow as tf
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
char_dataset

In [ ]:
sequence_length = 100
#تقطيع النص كله الي بيانات مقطعة
# build sequences by batching
sequences = char_dataset.batch(2*sequence_length + 1, drop_remainder=True)
sequences

In [55]:
for sequence in sequences.take(3):
  print(''.join([int2char[i] for i in sequence.numpy()]))

project gutenberg frankenstein mary wollstonecraft godwin shelley ebook use anyone anywhere cost almost restrictions whatsoever may copy give away use terms project gutenberg license included ebook onl
ine www gutenberg net title frankenstein modern prometheus author mary wollstonecraft godwin shelley release date june 17 2008 ebook 84 last updated january 13 2018 language english character set encod
ing utf 8 start project gutenberg ebook frankenstein produced judith boss christy phillips lynn hanninen david meltzer html version al haines corrections menno de leeuw frankenstein modern prometheus m


In [56]:
def split_sample(sample):
      ds = tf.data.Dataset.from_tensors((sample[:sequence_length], sample[sequence_length]))
      for i in range(1, (len(sample)-1) // 2):
        input_ = sample[i: i+sequence_length]
        target = sample[i+sequence_length]
        # extend the dataset with these samples by concatenate() method
        other_ds = tf.data.Dataset.from_tensors((input_, target))
        ds = ds.concatenate(other_ds)
      return ds
# prepare inputs and targets
dataset = sequences.flat_map(split_sample)

In [57]:
def one_hot_samples(input_, target):
    # onehot encode the inputs and the targets
    return tf.one_hot(input_, n_unique_chars), tf.one_hot(target, n_unique_chars)


dataset = dataset.map(one_hot_samples)

In [59]:
# print first 2 samples
for element in dataset.take(2):
    print("Input:", ''.join([int2char[np.argmax(char_vector)] for char_vector in element[0].numpy()]))
    print("Target:", int2char[np.argmax(element[1].numpy())])
    print("Input shape:", element[0].shape)
    print("Target shape:", element[1].shape)
    print("="*50, "\n")

Input: project gutenberg frankenstein mary wollstonecraft godwin shelley ebook use anyone anywhere cost alm
Target: o
Input shape: (100, 43)
Target shape: (43,)

Input: roject gutenberg frankenstein mary wollstonecraft godwin shelley ebook use anyone anywhere cost almo
Target: s
Input shape: (100, 43)
Target shape: (43,)



In [62]:
#عمل خلط للبيانات و تقسيمها الي باشتات
# repeat, shuffle and batch the dataset
ds = dataset.repeat().shuffle(1024).batch(BATCH_SIZE, drop_remainder=True)

In [63]:
#بناء الموديل
model = Sequential([
    LSTM(256, input_shape=(sequence_length, n_unique_chars), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(n_unique_chars, activation="softmax"),
])

In [64]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(ds, steps_per_epoch=(len(encoded_text) - sequence_length) // BATCH_SIZE, epochs=4)

Epoch 1/4
2108/2108 [==============================] - 3206s 2s/step - loss: 2.1863
Epoch 2/4
2108/2108 [==============================] - 3133s 1s/step - loss: 1.6795
Epoch 3/4
2108/2108 [==============================] - 3070s 1s/step - loss: 1.4670
Epoch 4/4
2108/2108 [==============================] - 3081s 1s/step - loss: 1.3479


In [65]:
!pip install "git+https://github.com/tqdm/tqdm.git@devel#egg=tqdm"


  Cloning https://github.com/tqdm/tqdm.git (to revision devel) to /tmp/pip-install-yvk1y4bp/tqdm_8cd81d5a41174512afeee50a24aea2f1
  Running command git clone --filter=blob:none --quiet https://github.com/tqdm/tqdm.git /tmp/pip-install-yvk1y4bp/tqdm_8cd81d5a41174512afeee50a24aea2f1
  Running command git checkout -b devel --track origin/devel
  Switched to a new branch 'devel'
  Branch 'devel' set up to track remote branch 'devel' from 'origin'.
  Resolved https://github.com/tqdm/tqdm.git to commit 6ce50bedb3c525f762258fb0701aacae15515f4d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tqdm: filename=tqdm-4.66.1.dev2+g6ce50be-py3-none-any.whl size=78482 sha256=f4cdf3b3ef820d0928f783aee2321d236f4ba5cbd7bbd1b85da4f74f1f94653b
  Stored in directory: /tmp/pip-ephem-wheel-cache-dxskfigf/wheels/58/37/2e/a68b55bdff1a302c7a4a688e0237765287e22abddc0f8382b8
Successfully built tqdm
  Attemptin

In [67]:
import tqdm
seed = "chapter xiii"
vocab_size = len(char2int)
s = seed
n_chars = 200
# generate 400 characters
generated = ""
for i in tqdm.tqdm(range(n_chars), "Generating text"):
    # make the input sequence
    X = np.zeros((1, sequence_length, vocab_size))
    for t, char in enumerate(seed):
        X[0, (sequence_length - len(seed)) + t, char2int[char]] = 1
    # predict the next character
    predicted = model.predict(X, verbose=0)[0]
    # converting the vector to an integer
    next_index = np.argmax(predicted)
    # converting the integer to a character
    next_char = int2char[next_index]
    # add the character to results
    generated += next_char
    # shift seed and the predicted character
    seed = seed[1:] + next_char

print("Seed:", s)
print("Generated text:")
print(generated)

Generating text: 100%|██████████| 200/200 [00:17<00:00, 11.45it/s]

Seed: chapter xiii
Generated text:
n state assure seas compliance endeavoured commance enterprise misery courage accomplish enterprise misery courage accomplish enterprise misery courage accomplish enterprise misery courage accomplish 


In [ ]:
from transformers.integrations import NeptuneCallback

# Create Neptune callback
neptune_callback = NeptuneCallback(
    name="DistilBERT",
    description="DistilBERT fine-tuned on GLUE/MRPC",
    tags=["args-callback", "fine-tune", "MRPC"],  # tags help you manage runs
    base_namespace="custom_name",  # the default is "finetuning"
    log_checkpoints="best",  # other options are "last", "same", and None
    capture_hardware_metrics=False,  # additional kwargs for a Neptune run
)

# Create training arguments
training_args = TrainingArguments(
    "quick-training-distilbert-mrpc",
    evaluation_strategy="steps",
    eval_steps=20,
    report_to="none",
)

# Pass Neptune callback to Trainer
trainer = Trainer(
    model,
    training_args,
    callbacks=[neptune_callback],
)

trainer.train()


In [ ]:
len(processed_inputs)

In [ ]:
processed_inputs[:2000]

In [14]:
chars = sorted(list(set(processed_inputs)))

In [15]:
len(chars)

43

In [16]:
chars[:15]

[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '_', 'a', 'b', 'c']

In [17]:
char_to_num = dict((c, i) for i, c in enumerate(chars))
char_to_num

{' ': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 '_': 11,
 'a': 12,
 'b': 13,
 'c': 14,
 'd': 15,
 'e': 16,
 'f': 17,
 'g': 18,
 'h': 19,
 'i': 20,
 'j': 21,
 'k': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'q': 28,
 'r': 29,
 's': 30,
 't': 31,
 'u': 32,
 'v': 33,
 'w': 34,
 'x': 35,
 'y': 36,
 'z': 37,
 'æ': 38,
 'è': 39,
 'é': 40,
 'ê': 41,
 'ô': 42}

In [18]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 269995
Total vocab: 43


In [19]:
seq_length = 100
x_data = []
y_data = []

# loop through inputs, start at the beginning and go until we hit
# the final character we can create a sequence out of
for i in range(0, input_len - seq_length, 1):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [20]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 269895


In [23]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X.shape

(269895, 100, 1)

In [ ]:
X = X/float(vocab_len)

In [ ]:
pip install keras.utils


In [ ]:
from keras.utils.np_utils import to_categorical



In [ ]:
y = np_utils.to_categorical(y_data)